# Diagonal Augment

Given a tensor and $n$, arrange the tensor so that the width is increased by $n$, creating a triangle of padded zeros in the upper left and lower right corners.

[[1,2,3,]

  [4,5,6],

  [7,8,9]]

n = 2 →

[[0,0,1,2,3],

 [0,4,5,6,0],

 [7,8,9,0,0]]

In [35]:
import torch

In [36]:
def diagonal_augment(t: torch.Tensor, n: int) -> torch.Tensor:
    pass

In [38]:
t = torch.tensor([[1,2,3], [4,5,6], [7,8,9]])

In [73]:
new_t = torch.nn.ZeroPad2d((0, 2, 0, 0))(t)

In [81]:
new_t

tensor([[1, 2, 3, 0, 0],
        [4, 5, 6, 0, 0],
        [7, 8, 9, 0, 0]])

In [83]:
torch.gather(new_t, 1, torch.tensor([3, 4, 0, 1, 2]))

RuntimeError: Index tensor must have the same number of dimensions as input tensor

In [74]:
new_t

tensor([[1, 2, 3, 0, 0],
        [4, 5, 6, 0, 0],
        [7, 8, 9, 0, 0]])

In [80]:
new_t.roll(shifts=(2, 1, 0), dims=(1, 1, 1))

tensor([[3, 0, 0, 1, 2],
        [6, 0, 0, 4, 5],
        [9, 0, 0, 7, 8]])

In [39]:
mask = torch.empty(3, 5)

In [40]:
mask

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [41]:
x_coords, y_coords = torch.meshgrid(torch.arange(3), torch.arange(5), indexing="ij")

In [44]:
x_coords

tensor([[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2]])

In [55]:
mask = (x_coords + y_coords < 2) + (2 - x_coords + 4 - y_coords < 2)

In [63]:
t

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [65]:
t[torch.arange(2), torch.arange(2)]

tensor([1, 5])

In [62]:
torch.where(mask, 0, t[x_coords, y_coords])

IndexError: index 3 is out of bounds for dimension 1 with size 3

In [56]:
new_t = torch.empty(3, 5)

In [58]:
new_t[:, 2:] = t

In [61]:
t.masked_fill(mask, 0)

RuntimeError: The size of tensor a (5) must match the size of tensor b (3) at non-singleton dimension 1

In [57]:
new_t.masked_fill(mask, t)

RuntimeError: masked_fill_ only supports a 0-dimensional value tensor, but got tensor with 2 dimension(s).

In [47]:
y_coords < 2 or y_coords 

tensor([[ True,  True, False, False, False],
        [ True,  True, False, False, False],
        [ True,  True, False, False, False]])

In [3]:
x = torch.arange(8).view(2, 2, 2)

In [5]:
x

tensor([[[0, 1],
         [2, 3]],

        [[4, 5],
         [6, 7]]])

In [6]:
x.flip(0)

tensor([[[4, 5],
         [6, 7]],

        [[0, 1],
         [2, 3]]])

In [7]:
x.flip(1)

tensor([[[2, 3],
         [0, 1]],

        [[6, 7],
         [4, 5]]])

In [8]:
x.flip(2)

tensor([[[1, 0],
         [3, 2]],

        [[5, 4],
         [7, 6]]])

In [9]:
x.flip(0) + x.flip(1) + x.flip(2)

tensor([[[ 7,  8],
         [ 9, 10]],

        [[11, 12],
         [13, 14]]])

In [11]:
torch.vstack([x.flip(0), x.flip(1), x.flip(2)])

tensor([[[4, 5],
         [6, 7]],

        [[0, 1],
         [2, 3]],

        [[2, 3],
         [0, 1]],

        [[6, 7],
         [4, 5]],

        [[1, 0],
         [3, 2]],

        [[5, 4],
         [7, 6]]])

In [17]:
x.unsqueeze(-1).permute(2, 0, 1, 3)

tensor([[[[0],
          [2]],

         [[4],
          [6]]],


        [[[1],
          [3]],

         [[5],
          [7]]]])

In [29]:
torch.tensor([2, 3, 4])[torch.randperm(3)]

tensor([2, 4, 3])

In [28]:
torch.randperm(3)

tensor([1, 0, 2])

In [33]:
torch.tensor([[2, 3, 4], [2, 3, 4]])[1, 2]

tensor(4)

In [34]:
torch.tensor([[2, 3, 4], [2, 3, 4]])[1][2]

tensor(4)

In [30]:
def permute_rows(tensor):
    # Create a tensor to store the permuted rows
    permuted_tensor = tensor.clone()
    for i in range(tensor.shape[0]):  # Iterate over rows
        permuted_tensor[i] = tensor[i, torch.randperm(tensor.shape[1])]
    return permuted_tensor

# Example usage
tensor = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
result = permute_rows(tensor)
print("Original Tensor:\n", tensor)
print("Permuted Tensor:\n", result)

Original Tensor:
 tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
Permuted Tensor:
 tensor([[2, 3, 1],
        [4, 6, 5],
        [9, 7, 8]])


In [60]:
import torch

# Original tensor and n
tensor = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
n = 2

# Compute new dimensions
rows, cols = tensor.shape
new_cols = cols + n

# Initialize a larger tensor with zeros
result = torch.zeros((rows, new_cols), dtype=tensor.dtype)

# Copy original values into the correct positions
result[:, n:] = tensor

# Create a mask for the upper-left and lower-right corners
mask = torch.zeros_like(result, dtype=torch.bool)

for i in range(rows):
    mask[i, :n - i] = True  # Upper-left corner
    mask[i, cols + (i - (rows - 1)):] = True  # Lower-right corner

# Apply mask to set padded values to zero
result = result.masked_fill(mask, 0)

print(result)


tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 7, 0, 0]])
